# 0.   Packages and Functions

In [1]:
# !pip install jupyter_nbextensions_configurator

In [2]:
# !jupyter nbextensions_configurator enable --user

In [3]:
 !jupyter nbextension enable nbextensions_configurator/config_menu/main

Enabling notebook extension nbextensions_configurator/config_menu/main...
      - Validating: problems found:
        - require?  X nbextensions_configurator/config_menu/main


In [4]:
!pip install pandas_datareader

In [1]:

#general
import pandas as pd
import numpy as np
import pickle 
from decimal import Decimal
import re


#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [2]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [3]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [4]:
horizons=[7,15,30,60,90,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [5]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [6]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [7]:
tech_c=['AAPL','AMZN','MSFT','GOOG','FB','VISA','NVDA']
auto_c=['F','GM','HMC','TSLA','TYO','HOG']
retail_c=[ 'WMT', 'M','TGT','KSS','COST','BBY']
fmcg_c=['NESN','KO','PEP','ULVR','PG', 'OR']
bank_c=['AXP','VISA','SC', 'DB', 'UBS','BAC','JPM']

In [8]:
#ESG ratios
ESG_ratios=[16.5, 27.4, 14.7, 22.8, 31.6, 12.9, 29.7, 30.6, 28.5, 31.1, 30.3, 16.3, 27.3, 14.5, 14.8, 12.9, 22.3, 12.0, 28.3, 25.1, 17.6, 23.4, 25.1, 20.5, 19.8, 17.4, 26.9, 30.0, 24.8, 26.3, 27.9]

We are going to extract info from 10 years from the companies selected

In [9]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()
error_indexes=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)
        
        error_company_index=companies.index(company)
        error_indexes.append(error_company_index)


for c in error_companies:
  companies.remove(c)

ESG_ratios_fix=list()

for index in range(0, len(ESG_ratios)):
    if index not in error_indexes:
        ESG_ratios_fix.append(ESG_ratios[index])

AAPL
AMZN
MSFT
GOOG
FB
NVDA
F
GM
HMC
TSLA
TYO
HOG
WMT
M
TGT
KSS
COST
BBY
There were  errors when extracting data of   NESN
KO
PEP
There were  errors when extracting data of   ULVR
PG
OR
AXP
There were  errors when extracting data of   VISA
SC
DB
UBS
BAC
JPM


In [10]:
companies

['AAPL',
 'AMZN',
 'MSFT',
 'GOOG',
 'FB',
 'NVDA',
 'F',
 'GM',
 'HMC',
 'TSLA',
 'TYO',
 'HOG',
 'WMT',
 'M',
 'TGT',
 'KSS',
 'COST',
 'BBY',
 'KO',
 'PEP',
 'PG',
 'OR',
 'AXP',
 'SC',
 'DB',
 'UBS',
 'BAC',
 'JPM']

In [11]:
len(ESG_ratios_fix)

28

In [12]:
len(companies)

28

In [13]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [14]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

# 3. Random transactions

In [15]:
transactions=pd.DataFrame()
#we do 500.000 initial transactions

#id
transactions['id']=range(1,200001)

## Companies

In [16]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Sector

In [17]:
#sector of the company

random_companies=transactions['company'].tolist()
random_sectors=list()
for comp in random_companies:
    if comp in tech_c:
        sector='TECH'
    elif comp in auto_c:
        sector='AUTO'
    elif comp in retail_c:
        sector='RETAIL'
    elif comp in fmcg_c:
        sector='FMCG'
    elif comp in bank_c:
        sector='BANK'
    
    random_sectors.append(sector)

transactions['sector']=random_sectors


## Horizons

In [18]:
#random horizon
transactions["horizon (days)"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [19]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [20]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

200000


### Fixing dates (BUY)

In [21]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [22]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY


In [23]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [24]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Expected return and volatility

### Expected volatilities

In [25]:

l_volatilities = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    log_ret = np.log(df_company_prices['Close'] / df_company_prices['Close'].shift(1))
    volatility = log_ret.rolling(window=252).std() * np.sqrt(252)
    df_company_prices.insert(2, "Volatility", volatility, True)
    l_volatilities.append(df_company_prices)

print(l_volatilities)

2  
2010-10-14   9213600.0  16.657616  
2010-10-15   6672800.0  16.754620  
...                ...        ...  
2020-10-05  14631000.0   6.060000  
2020-10-06  22708200.0   5.810000  
2020-10-07  17965500.0   5.960000  
2020-10-08  21298800.0   6.290000  
2020-10-09  19317400.0   6.230000  

[2518 rows x 7 columns],                   High         Low  Volatility        Open       Close  \
Date                                                                     
2010-10-11   55.000000   54.240002         NaN   54.299999   54.849998   
2010-10-12   54.910000   54.470001         NaN   54.869999   54.740002   
2010-10-13   55.080002   54.349998         NaN   54.689999   54.419998   
2010-10-14   54.619999   53.520000         NaN   54.459999   54.070000   
2010-10-15   54.590000   53.970001         NaN   54.400002   54.330002   
...                ...         ...         ...         ...         ...   
2020-10-05  162.309998  159.990005    0.391384  159.990005  161.490005   
2020-10-06  161.

### Expected returns

In [26]:
l_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    returns = (df_company_prices['Close'] - df_company_prices['Close'].shift(1)) / df_company_prices['Close'].shift(1)
    df_company_prices.insert(2, "returns", returns, True)
    l_returns.append(df_company_prices)

In [27]:
l_returns

3.240002  51.970001  0.016052         NaN  52.810001  53.169998   
 ...               ...        ...       ...         ...        ...        ...   
 2020-10-05  20.750000  20.049999  0.023046    0.924426  20.330000  20.420000   
 2020-10-06  20.750000  19.360001 -0.045544    0.925006  20.750000  19.490000   
 2020-10-07  20.740000  19.830000  0.059005    0.927001  19.830000  20.639999   
 2020-10-08  21.900000  20.719999  0.060562    0.928778  20.870001  21.889999   
 2020-10-09  22.240000  21.430000 -0.016446    0.928351  22.000000  21.530001   
 
                Volume  Adj Close  
 Date                              
 2010-10-11  2392500.0  37.662571  
 2010-10-12  2589000.0  37.592255  
 2010-10-13  3001500.0  37.275810  
 2010-10-14  5694400.0  36.797657  
 2010-10-15  4613400.0  37.388332  
 ...               ...        ...  
 2020-10-05  6275500.0  20.420000  
 2020-10-06  7438600.0  19.490000  
 2020-10-07  6694300.0  20.639999  
 2020-10-08  8074000.0  21.889999  
 2020-10-09  

In [28]:
l_yearly_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    try:
        df_yearly = df_company_prices["Close"].resample('Y').first().pct_change()
        
        yearly_mean = df_yearly.mean()
    except:
        print( company)
        print(df_company_prices)
        yearly_mean = 0
    l_yearly_returns.append([company,yearly_mean])
print(l_yearly_returns)

[['AAPL', 0.24576212706845474], ['AMZN', 0.32878772362638664], ['MSFT', 0.21823741952413256], ['GOOG', 0.191378431933231], ['FB', 0.30161872761801845], ['NVDA', 0.4931171561846849], ['F', -0.011976915482311935], ['GM', 0.04088241949150044], ['HMC', -0.009170559827996482], ['TSLA', 0.5157369284319866], ['TYO', -0.10592633571589997], ['HOG', 0.044568829075491104], ['WMT', 0.09703322358290575], ['M', 0.008681139192578402], ['TGT', 0.11412148248236709], ['KSS', 0.006854575724160139], ['COST', 0.1674207351165724], ['BBY', 0.2622984487062406], ['KO', 0.06441514580167561], ['PEP', 0.07940352791222091], ['PG', 0.07725201668740694], ['OR', -0.06175039426047935], ['AXP', 0.14723187952081127], ['SC', 0.011489650389287712], ['DB', -0.14093138981130585], ['UBS', -0.0394967250395171], ['BAC', 0.18910217427914794], ['JPM', 0.15206171805688737]]


## Prices - BUY

In [29]:
#price when bought

l_buy_prices=list()
l_volatility=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    try:
        price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
        volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
    except :
        price=np.NaN

    l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


In [30]:
l_volatility_buy=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    try:
        volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
    except :
        volatility=np.NaN
    
    l_volatility_buy.append(volatility)
    
print(len(l_volatility_buy))


transactions['Volatility_Buy'] = l_volatility_buy

190092


## Dates - SELL

In [31]:
#random sell date
dates_SELL=list()
expected_return = list()

for index,row in transactions.iterrows():
    comp=row['company']
    company_pos=companies.index(comp)
    buy_date = row['date_BUY_fix']
    sell_date = buy_date + datetime.timedelta(row['horizon (days)'])
    expected_return.append(row['horizon (days)']/365 * l_yearly_returns[company_pos][1])

    dates_SELL.append(sell_date)

transactions['expected_return (yearly)'] = expected_return
transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [32]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [33]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

### Sharpe Ratio 

In [34]:
l_sharpe=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    r = l_returns
    try:
        df_company_returns = pd.DataFrame(l_returns[company_pos])
        r=df_company_returns["returns"]
        sr = r.mean()/r.std() * np.sqrt(252)
        
    except :
        sr=np.NaN
        
    l_sharpe.append(sr)
    
print(len(l_sharpe))


transactions['Sharpe Ratio'] = l_sharpe

190092


In [35]:
#checking for holidays (NYSE, NASDAQ is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [36]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=np.NaN

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [37]:
l_volatility_sell=list()
for index,row in transactions.iterrows():
    comp=row['company']
    sell_date=row['date_SELL_fix']
    company_pos=companies.index(comp)
    try:
        volatility=l_volatilities[company_pos].loc[sell_date,'Volatility']
    except :
        volatility=np.NaN

        
    l_volatility_sell.append(volatility)

transactions['Volatility_sell']=l_volatility_sell

In [38]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']
transactions=transactions[transactions.holiday_SELL!='Holiday']

# Dataset transactions - review missing values

In [39]:
transactions

,id,company,sector,horizon (days),amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,...,Volatility_Buy,expected_return (yearly),date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,Sharpe Ratio,holiday_SELL,price_SELL,Volatility_sell
0,1,TSLA,AUTO,7,500,2017-06-14,Wed,2017-06-14,Wed,,...,0.336212,0.009891,2017-06-21,Wed,2017-06-21,Wed,1.115540,,75.279999,0.336167
1,2,SC,BANK,600,3000,2018-07-09,Mon,2018-07-09,Mon,,...,0.291036,0.018887,2020-02-29,Sat,2020-02-28,Fri,0.132227,,23.415619,0.247294
2,3,DB,BANK,330,5000,2014-04-03,Thu,2014-04-03,Thu,,...,0.261244,-0.127417,2015-02-27,Fri,2015-02-27,Fri,-0.210016,,30.680777,0.260936
3,4,AMZN,TECH,90,15000,2015-01-10,Sat,2015-01-09,Fri,,...,0.329601,0.081071,2015-04-09,Thu,2015-04-09,Thu,1.129542,,383.540009,0.324660
5,6,BBY,RETAIL,30,50,2016-06-30,Thu,2016-06-30,Thu,,...,0.349990,0.021559,2016-07-30,Sat,2016-07-29,Fri,0.462873,,29.844707,0.348505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,KSS,RETAIL,60,50,2015-05-09,Sat,2015-05-08,Fri,,...,0.226325,0.001127,2015-07-07,Tue,2015-07-07,Tue,0.001703,,51.271896,0.263863
199996,199997,OR,FMCG,7,500,2014-08-16,Sat,2014-08-15,Fri,,...,NaN,-0.001184,2014-08-22,Fri,2014-08-22,Fri,0.158124,,NaN,NaN
199997,199998,AAPL,TECH,90,100,2016-04-14,Thu,2016-04-14,Thu,,...,0.275282,0.060599,2016-07-13,Wed,2016-07-13,Wed,0.997028,,22.695921,0.281343
199998,199999,DB,BANK,540,1000,2015-08-10,Mon,2015-08-10,Mon,,...,0.292055,-0.208501,2017-01-31,Tue,2017-01-31,Tue,-0.210016,,19.146446,0.517937


In [40]:
print(transactions.isnull().values.sum())
transactions.isnull().sum()


29172


id                              0
company                         0
sector                          0
horizon (days)                  0
amount                          0
date_BUY                        0
day_BUY                         0
date_BUY_fix                    0
day_BUY_fix                     0
holiday_BUY                     0
price_BUY                    6617
Volatility_Buy              10489
expected_return (yearly)        0
date_SELL                       0
day_SELL                        0
date_SELL_fix                   0
day_SELL_fix                    0
Sharpe Ratio                    0
holiday_SELL                    0
price_SELL                   4453
Volatility_sell              7613
dtype: int64

In [41]:
df_review=transactions.copy()
print('original    :',df_review.shape[0])
print('without NAN :',df_review.dropna().shape[0])
print('rows with NAN: ',df_review.shape[0]-df_review.dropna().shape[0])

original    : 181298
without NAN : 169720
rows with NAN:  11578


In [42]:
transactions=transactions.dropna()

In [43]:
transactions.shape

(169720, 21)

Now, getting only relevant columns

In [44]:
transactions_ready=transactions[['company','sector','horizon (days)','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL','Volatility_Buy','Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)']]

In [45]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly)
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559
...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501


# Evaluating investment performance 

## Inflation
As investors, we seek to at least beat the rate of inflation. 

In [46]:
inflation_ratios=pd.read_csv('./../datasets/ratios_inflacion.csv', index_col=False)
# inflation_ratios['Year'] = pd.to_datetime(inflation_ratios['Year'], format='%Y')

inflation_ratios=inflation_ratios.set_index('Year')

# inflation.info()

In [47]:
inflation_ratios.loc[2017,'Tasa_inflacion']

1.96

In [48]:
l_inflation=list()
for index,row in transactions_ready.iterrows():
    buy_date=row['date_BUY_fix']
    buy_date_str=str(buy_date)
    year=int(buy_date_str[:4])
    try:
        inflation=inflation_ratios.loc[year,'Tasa_inflacion']
    except :
        inflation=np.NaN

    l_inflation.append(inflation)

transactions_ready['inflation']=l_inflation


In [49]:
transactions_ready.isnull().sum

<bound method DataFrame.sum of         company  sector  horizon (days)  amount  date_BUY_fix  date_SELL_fix  \
0         False   False           False   False         False          False   
1         False   False           False   False         False          False   
2         False   False           False   False         False          False   
3         False   False           False   False         False          False   
5         False   False           False   False         False          False   
...         ...     ...             ...     ...           ...            ...   
199994    False   False           False   False         False          False   
199995    False   False           False   False         False          False   
199997    False   False           False   False         False          False   
199998    False   False           False   False         False          False   
199999    False   False           False   False         False          False   

        

In [50]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),inflation
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891,1.96
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887,1.68
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417,-0.15
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071,-0.50
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559,-0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937,-0.50
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127,-0.50
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599,-0.20
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501,-0.50


## Real Profit/Loss (Rentabilidad Real)

In [51]:
    transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']


In [52]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),inflation,nominal_return
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891,1.96,-0.011191
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887,1.68,0.329529
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417,-0.15,-0.218842
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071,-0.50,0.291685
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559,-0.20,0.098039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937,-0.50,-0.053193
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127,-0.50,-0.119221
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599,-0.20,-0.130599
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501,-0.50,-0.415100


In [53]:
transactions_ready.isnull().sum()

company                     0
sector                      0
horizon (days)              0
amount                      0
date_BUY_fix                0
date_SELL_fix               0
price_BUY                   0
price_SELL                  0
Volatility_Buy              0
Volatility_sell             0
Sharpe Ratio                0
expected_return (yearly)    0
inflation                   0
nominal_return              0
dtype: int64

In [54]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
    inflation=row['inflation']
    if inflation <0:
        if row['nominal_return']>0:
            inv='GOOD'
            investments.append(inv)
        elif row['nominal_return']<0:
            inv='BAD'
            investments.append(inv)
        else:
            inv='BAD'
            investments.append(inv)
    else:
        if row['nominal_return']>inflation:
            inv='GOOD'
            investments.append(inv)
        elif row['nominal_return']<inflation:
            inv='BAD'
            investments.append(inv)
        else:
            inv='BAD'
            investments.append(inv)
        

transactions_ready['investment']=investments

In [55]:
transactions_ready.shape

(169720, 15)

In [56]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),inflation,nominal_return,investment
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891,1.96,-0.011191,BAD
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887,1.68,0.329529,BAD
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417,-0.15,-0.218842,BAD
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071,-0.50,0.291685,GOOD
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559,-0.20,0.098039,GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937,-0.50,-0.053193,BAD
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127,-0.50,-0.119221,BAD
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599,-0.20,-0.130599,BAD
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501,-0.50,-0.415100,BAD


In [57]:
transactions_ready.investment.value_counts()

BAD     108210
GOOD     61510
Name: investment, dtype: int64

# Ratios

## ESG Ratios

In [58]:
len(ESG_ratios_fix)

28

In [59]:
companies[3]

'GOOG'

In [60]:
ESG_ratios_fix

[16.5,
 27.4,
 14.7,
 22.8,
 31.6,
 12.9,
 29.7,
 30.6,
 28.5,
 31.1,
 30.3,
 16.3,
 27.3,
 14.5,
 14.8,
 12.9,
 22.3,
 12.0,
 25.1,
 17.6,
 25.1,
 20.5,
 19.8,
 26.9,
 30.0,
 24.8,
 26.3,
 27.9]

In [61]:
transactions_ready['ESG_ranking']=0

for index, row in transactions_ready.iterrows():
    company=row['company']
    ind_comp=companies.index(company)
    ESG_ratio= ESG_ratios_fix[ind_comp]
    transactions_ready.loc[index, 'ESG_ranking']=ESG_ratio
            

In [62]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),inflation,nominal_return,investment,ESG_ranking
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891,1.96,-0.011191,BAD,31.1
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887,1.68,0.329529,BAD,26.9
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417,-0.15,-0.218842,BAD,30.0
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071,-0.50,0.291685,GOOD,27.4
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559,-0.20,0.098039,GOOD,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937,-0.50,-0.053193,BAD,14.7
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127,-0.50,-0.119221,BAD,12.9
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599,-0.20,-0.130599,BAD,16.5
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501,-0.50,-0.415100,BAD,30.0


In [65]:
pe_ratios_collection= pickle.load(open('./../webscrapping/pe_ratios_collection.p', "rb" ))

In [66]:
valor=pe_ratios_collection['AAPL'][pe_ratios_collection['AAPL']['Date']=='2020-09'].PE_Ratio.values[0]
float(valor)

35.46

In [67]:
#getting nearest date before investment.
def nearest(items, pivot):
    return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [68]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PE_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].PE_Ratio.values[0]
            PE_ratio=float(PE_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PE_ratio=np.NaN
            
    except:
        PE_ratio=Decimal('nan')
    
    l_PE_ratios.append(PE_ratio)

In [69]:
transactions_ready['PE_ratio']=l_PE_ratios

In [70]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),inflation,nominal_return,investment,ESG_ranking,PE_ratio
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,1.115540,0.009891,1.96,-0.011191,BAD,31.1,0
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,0.132227,0.018887,1.68,0.329529,BAD,26.9,4.65
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,-0.210016,-0.127417,-0.15,-0.218842,BAD,30.0,599.56
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,1.129542,0.081071,-0.50,0.291685,GOOD,27.4,0
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,0.462873,0.021559,-0.20,0.098039,GOOD,12.0,9.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,0.977717,0.017937,-0.50,-0.053193,BAD,14.7,36.29
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,0.001703,0.001127,-0.50,-0.119221,BAD,12.9,13.03
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,0.997028,0.060599,-0.20,-0.130599,BAD,16.5,11.29
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,-0.210016,-0.208501,-0.50,-0.415100,BAD,30.0,26.28


## EPS Ratio

In [88]:
import re

In [89]:
l_EPS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']

    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),row['date_BUY_fix'])
    
        try:
            EPS_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].TTM_Net_EPS.values[0]
            EPS_ratio_str=re.sub('[%$B]', '',EPS_ratio_str)
        
            EPS_ratio=float(EPS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            EPS_ratio=np.NaN
            
    except:
        EPS_ratio=np.NaN
    
    l_EPS_ratios.append(EPS_ratio)

In [90]:
transactions_ready['EPS_ratio']=l_EPS_ratios

## PS Ratio

In [71]:
ps_ratios_collection= pickle.load(open('./../webscrapping/ps_ratios_collection.p', "rb" ))


In [72]:
l_PS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=ps_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PS_ratio_str=ps_ratios_collection[company][ps_ratios_collection[company]['Date']==date].Price_to_Sales_Ratio.values[0]
            PS_ratio=float(PS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PS_ratio= np.NaN
            
    except:
        PS_ratio=np.NaN
    
    l_PS_ratios.append(PS_ratio)

    
transactions_ready['PS_ratio']=l_PS_ratios

## PB Ratio

In [73]:
pb_ratios_collection= pickle.load(open('./../webscrapping/pb_ratios_collection.p', "rb" ))

In [74]:
l_PB_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pb_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PB_ratio_str=pb_ratios_collection[company][pb_ratios_collection[company]['Date']==date].Price_to_Book_Ratio.values[0]
            PB_ratio=float(PB_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PB_ratio= np.NaN
            
    except:
        PB_ratio=np.NaN
    
    l_PB_ratios.append(PB_ratio)

    
transactions_ready['PB_ratio']=l_PB_ratios

## NetProfitMargin ratio 

In [75]:
NetProfitMargin_ratios_collection= pickle.load(open('./../webscrapping/NetProfitMargin_ratios_collection.p', "rb" ))

In [76]:
l_NetProfitMargin_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=NetProfitMargin_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
        try:                       
            NetProfitMargin_ratio_str=NetProfitMargin_ratios_collection[company][NetProfitMargin_ratios_collection[company]['Date']==date].Net_Margin.values[0]
            NetProfitMargin_ratio_str=re.sub('[%$B]', '',NetProfitMargin_ratio_str)
            NetProfitMargin_ratio=float(NetProfitMargin_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            NetProfitMargin_ratio= np.NaN        
    except:
        NetProfitMargin_ratio=np.NaN
    
    l_NetProfitMargin_ratios.append(NetProfitMargin_ratio)
transactions_ready['NetProfitMargin_ratio']=l_NetProfitMargin_ratios

## Current Ratio


In [77]:
current_ratios_collection= pickle.load(open('./../webscrapping/current_ratios_collection.p', "rb" ))
current_ratios_collection['AAPL'].head()

,Date,Current_Assets,Current_Liabilities,Current_Ratio
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$143.71B,$105.39B,1.36
3,2020-06-01,$140.07B,$95.32B,1.47
4,2020-03-01,$143.75B,$96.09B,1.50


In [78]:
l_current_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=current_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            current_ratio_str=current_ratios_collection[company][current_ratios_collection[company]['Date']==date].Current_Ratio.values[0]
            current_ratio=float(current_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            current_ratio= np.NaN
            
    except:
        current_ratio=np.NaN
    
    l_current_ratios.append(current_ratio)

    
transactions_ready['current_ratio']=l_current_ratios

## ROA Ratio

In [79]:
roa_ratios_collection= pickle.load(open('./../webscrapping/roa_ratios_collection.p', "rb" ))
roa_ratios_collection['AAPL'].head()

,Date,TTM_Net_Income,Total_Assets,Return_on_Assets
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$57.41B,$323.89B,17.63%
3,2020-06-01,$58.42B,$317.34B,17.75%
4,2020-03-01,$57.22B,$320.40B,17.31%


In [80]:
l_roa_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roa_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roa_ratio_str=roa_ratios_collection[company][roa_ratios_collection[company]['Date']==date].Return_on_Assets.values[0]
            roa_ratio_str=re.sub('[%$B]','',roa_ratio_str)
            roa_ratio=float(roa_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roa_ratio= np.NaN
            
    except:
        roa_ratio=np.NaN
    
    l_roa_ratios.append(roa_ratio)

    
transactions_ready['roa_ratio']=l_roa_ratios

## ROE Ratio

In [81]:
roe_ratios_collection= pickle.load(open('./../webscrapping/roe_ratios_collection.p', "rb" ))
roe_ratios_collection['AAPL'].head()

,Date,TTM_Net_Income,Shareholder's_Equity,Return_on_Equity
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$57.41B,$65.34B,75.15%
3,2020-06-01,$58.42B,$72.28B,70.66%
4,2020-03-01,$57.22B,$78.43B,64.49%


In [82]:
l_roe_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roe_ratio_str=roe_ratios_collection[company][roe_ratios_collection[company]['Date']==date].Return_on_Equity.values[0]
            roe_ratio_str=re.sub('[%$B]','',roe_ratio_str)
            roe_ratio=float(roe_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roe_ratio= np.NaN
            
    except:
        roe_ratio=np.NaN
    
    l_roe_ratios.append(roe_ratio)

    
transactions_ready['roe_ratio']=l_roe_ratios

# Final dataset review 

In [83]:
transactions_ready

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,...,nominal_return,investment,ESG_ranking,PE_ratio,PS_ratio,PB_ratio,NetProfitMargin_ratio,current_ratio,roa_ratio,roe_ratio
0,TSLA,AUTO,7,500,2017-06-14,2017-06-21,76.132004,75.279999,0.336212,0.336167,...,-0.011191,BAD,31.1,0,5.65,9.71,-7.60,0.97,-3.54,-15.13
1,SC,BANK,600,3000,2018-07-09,2020-02-28,17.611959,23.415619,0.291036,0.247294,...,0.329529,BAD,26.9,4.65,0.87,0.89,18.62,0.87,3.38,20.59
2,DB,BANK,330,5000,2014-04-03,2015-02-27,39.276001,30.680777,0.261244,0.260936,...,-0.218842,BAD,30.0,599.56,0.79,0.56,0.34,0.73,0.01,0.25
3,AMZN,TECH,90,15000,2015-01-09,2015-04-09,296.929993,383.540009,0.329601,0.324660,...,0.291685,GOOD,27.4,0,1.62,13.44,-0.27,1.12,-0.57,-2.29
5,BBY,RETAIL,30,50,2016-06-30,2016-07-29,27.180000,29.844707,0.349990,0.348505,...,0.098039,GOOD,12.0,9.65,0.25,2.06,2.53,1.47,7.23,22.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,MSFT,TECH,30,4000,2015-12-10,2016-01-08,50.435631,47.752792,0.281871,0.281726,...,-0.053193,BAD,14.7,36.29,4.70,5.24,13.29,3.00,6.65,14.44
199995,KSS,RETAIL,60,50,2015-05-08,2015-07-07,58.212013,51.271896,0.226325,0.263863,...,-0.119221,BAD,12.9,13.03,0.60,1.88,4.56,1.91,6.01,14.72
199997,AAPL,TECH,90,100,2016-04-14,2016-07-13,26.105249,22.695921,0.275282,0.281343,...,-0.130599,BAD,16.5,11.29,2.53,4.26,22.27,1.28,17.44,40.24
199998,DB,BANK,540,1000,2015-08-10,2017-01-31,32.734558,19.146446,0.292055,0.517937,...,-0.415100,BAD,30.0,26.28,0.74,0.48,3.49,0.71,0.09,2.09


In [92]:
print(transactions_ready.isnull().values.sum())
transactions_ready.isnull().sum()


0


company                     0
sector                      0
horizon (days)              0
amount                      0
date_BUY_fix                0
date_SELL_fix               0
price_BUY                   0
price_SELL                  0
Volatility_Buy              0
Volatility_sell             0
Sharpe Ratio                0
expected_return (yearly)    0
inflation                   0
nominal_return              0
investment                  0
ESG_ranking                 0
PE_ratio                    0
PS_ratio                    0
PB_ratio                    0
NetProfitMargin_ratio       0
current_ratio               0
roa_ratio                   0
roe_ratio                   0
EPS_ratio                   0
dtype: int64

In [93]:
df_review=transactions_ready.copy()
print('original    :',df_review.shape[0])
print('without NAN :',df_review.dropna().shape[0])
print('rows with NAN: ',df_review.shape[0]-df_review.dropna().shape[0])

original    : 161898
without NAN : 161898
rows with NAN:  0


In [94]:
transactions_ready=transactions_ready.dropna()
transactions_ready.shape

(161898, 24)

In [95]:
transactions_ready.to_csv('transactions_variables.csv')